# Rain in Australia - Next-Day Prediction Model
---

Student name: T.J. Kyner \
Student pace: Full time \
Instructor: Abhineet Kulkarni \
Cohort: 040521

## Business Problem
---

## Imports & Settings
---

## Exploratory Data Analysis
---

### Column Definitions

### Exploration

## Modeling
---

### 1. Baseline Model

### 2. 

### 3. 

## Conclusion
---

### Results

### Next Steps